<a href="https://colab.research.google.com/github/dipta007/Movie-Recommendation/blob/multi-label-classification/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from os.path import join
import os

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/Colab_data/MR1'       # path to your project on Drive
drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   
%cd "{PROJECT_PATH}"
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Colab_data/MR1’: File exists
/content/drive/My Drive/Colab_data/MR1
data/  model  multilayer_perceptron_graph.png


In [61]:
%matplotlib inline
import numpy as np
import pandas as pd

DATA_PATH = './data/ml-latest-small'

ratings = pd.read_csv(f"{DATA_PATH}/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [62]:
movies = pd.read_csv(f"{DATA_PATH}/movies.csv")
movies_ind_to_name = dict(zip(movies.movieId, movies.title))
movies_name_to_ind = dict(zip(movies.title, movies.movieId))
movies_ind_to_genre = dict(zip(movies.movieId, movies.genres))
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
maxLen = max(movies['genres'].apply(lambda x: len(x.split('|'))))

def make_all_same(x):
  now = x.split("|")
  while len(now) < maxLen:
    now.append("")
  return now

movies['genres_list'] = movies['genres'].apply(make_all_same)

In [64]:
s = set()
for v in movies['genres_list'].values:
  for g in v:
    s.add(g)

n_genres = len(s)
n_genres


21

In [65]:
joined_ratings = pd.merge(ratings, movies, "inner", "movieId")
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres,genres_list
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fanta..."
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fanta..."
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fanta..."
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fanta..."
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fanta..."


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

user_enc = LabelEncoder()
joined_ratings['user'] = user_enc.fit_transform(joined_ratings['userId'].values)
n_users = joined_ratings['user'].nunique()

movie_enc = LabelEncoder()
movie_enc.fit(movies['movieId'])
joined_ratings['movie'] = movie_enc.transform(joined_ratings['movieId'].values)
n_movies = movies['movieId'].nunique()

joined_ratings['rating'] = joined_ratings['rating'].values.astype(np.float32)
min_rating = min(joined_ratings['rating'])
max_rating = max(joined_ratings['rating'])

# n_users, n_movies, min_rating, max_rating
joined_ratings.head()
joined_ratings.shape

(100836, 9)

In [67]:
genre_enc = LabelEncoder()
genre_enc.fit(list(s))
joined_ratings['genres_list'] = joined_ratings['genres_list'].apply(lambda x: genre_enc.transform(x))
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres,genres_list,user,movie
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",0,0
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",4,0
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",6,0
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",14,0
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",16,0


In [68]:
n_users, n_movies, n_genres

(610, 9742, 21)

In [69]:
# Delete all the rows with less ratings
indexes = joined_ratings[ joined_ratings.rating < 3.5 ].index
joined_ratings = joined_ratings.drop(indexes)
joined_ratings.shape

(61716, 9)

In [70]:
final_one = joined_ratings.drop(['timestamp', 'genres'], axis=1)
final_one.head()

,userId,movieId,rating,title,genres_list,user,movie
0,1,1,4.0,Toy Story (1995),"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",0,0
1,5,1,4.0,Toy Story (1995),"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",4,0
2,7,1,4.5,Toy Story (1995),"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",6,0
4,17,1,4.5,Toy Story (1995),"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",16,0
5,18,1,3.5,Toy Story (1995),"[3, 4, 5, 6, 10, 0, 0, 0, 0, 0]",17,0


In [71]:
del joined_ratings
del ratings
del nratings
import gc
gc.collect()

6371

In [0]:
user_movie = {}
for index, row in final_one.iterrows():
  now = int(row.user)
  if now not in user_movie:
    user_movie[now] = []
  user_movie[now].append((row.movie, row.rating, row.genres_list))

MIN_NUMBER_OF_MOVIES = 4

In [73]:
MIN_NUMBER_OF_MOVIES, n_users, n_movies, n_genres, min_rating, max_rating

(4, 610, 9742, 21, 0.5, 5.0)

In [0]:
def convert_to_one_hot(movies):
  now = [0 for _ in range(n_movies)]
  for movie, rating, genre in movies:
    now[int(movie)] = 1.0
  return now

prev_movies = []
prev_genres = []
next_movies = []
for user in range(n_users):
  if user not in user_movie:
    continue
  user_movie[user].sort(key=lambda x: x[1])
  for i in range(len(user_movie[user]) - MIN_NUMBER_OF_MOVIES - 1):
    now_movies = user_movie[user]
    
    # Previous movies he has saw already
    prev_m = [mov for (mov, rat, genre) in now_movies[i:i+MIN_NUMBER_OF_MOVIES]]
    prev_g = [genre for (mov, rat, genre) in now_movies[i:i+MIN_NUMBER_OF_MOVIES]]
    prev_movies.append(prev_m);
    prev_genres.append(prev_g);

    # Next movies he should see
    next_movies.append(convert_to_one_hot(now_movies[i+MIN_NUMBER_OF_MOVIES:]));


In [76]:
len(prev_movies), len(next_movies)

(58682, 58682)

In [117]:
X_m = np.array(prev_movies).reshape(len(prev_movies), 4, 1)
X_g = np.array(prev_genres).reshape(len(prev_genres), MIN_NUMBER_OF_MOVIES * 10)
# X = np.concatenate()
X_m.shape, X_g.shape
Y = np.array(next_movies)
X.shape, Y.shape

((58682, 4, 2), (58682, 9742))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=47)

In [81]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((52813, 4, 2), (5869, 4, 2), (52813, 9742), (5869, 9742))

In [111]:
X_train[0][:][0], X_train[0][0][1]
# X_train_array = [X_train[:][:][0], X_train[:][:][1]]
# X_test_array = [X_test[:][:][0].reshape(4, 1), X_test[:][:][1]]

(array([2605, array([ 6,  9, 16,  0,  0,  0,  0,  0,  0,  0])],
       dtype=object), array([ 6,  9, 16,  0,  0,  0,  0,  0,  0,  0]))

In [0]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2
from keras.layers import Concatenate, Dense, Dropout, Activation, Lambda, LSTM, Flatten

def Model1(n_users, n_movies):
  movie = Input(shape=(MIN_NUMBER_OF_MOVIES, 1,))
  # m = LSTM(128)(movie)

  genre = Input(shape=(MIN_NUMBER_OF_MOVIES * 10, ))
  g = Embedding(n_genres, 20)(genre)
  g = Flatten()(g)

  # x = Concatenate()([g])
  x = g
  x = Dense(1000)(x)
  x = Dropout(0.2)(x)
  x = Dense(n_movies)(x)
  x = Activation('sigmoid')(x)

  model = Model(inputs=[movie, genre], outputs=x)
  # rms = RMSprop(lr=0.001)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  
  # model.compile(loss='mean_squared_error', optimizer=rms, metrics=['accuracy'])
  return model

In [151]:
model = Model1(n_users, n_movies)
model.summary()

# from keras.utils import plot_model
# plot_model(model, to_file='multilayer_perceptron_graph.png')

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 40)                0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 40, 20)            420       
_________________________________________________________________
flatten_9 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1000)              801000    
_________________________________________________________________
dropout_10 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 9742)              9751742   
_________________________________________________________________
activation_10 (Activation)   (None, 9742)              0   

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

CHECKPOINT_PATH = './model-multi-label'

callbacks = []
callbacks.append(EarlyStopping(monitor='val_loss', patience=2))
callbacks.append(ModelCheckpoint(CHECKPOINT_PATH,  monitor='val_loss', save_best_only=True))

In [153]:
history = model.fit(x=[X_m, X_g], y=Y, batch_size=128, epochs=50,
                    verbose=1, validation_split=0.1,
                    callbacks=callbacks)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 52813 samples, validate on 5869 samples
Epoch 1/50
52813/52813 [==============================] - 21s 393us/step - loss: 0.0814 - accuracy: 0.9815 - val_loss: 0.1155 - val_accuracy: 0.9749
Epoch 2/50
52813/52813 [==============================] - 21s 392us/step - loss: 0.0632 - accuracy: 0.9838 - val_loss: 0.1167 - val_accuracy: 0.9749
Epoch 3/50
52813/52813 [==============================] - 20s 386us/step - loss: 0.0628 - accuracy: 0.9838 - val_loss: 0.1175 - val_accuracy: 0.9749


In [155]:
from keras.models import load_model

best = load_model(CHECKPOINT_PATH)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
def get_result(movie_nos, n = 10):
  movies = movie_enc.transform(movie_nos)
  genres = np.array([list(genre_enc.transform(make_all_same(movies_ind_to_genre[movie_id]))) for movie_id in movie_nos]).reshape(1, MIN_NUMBER_OF_MOVIES * 10)
  # print(genres)
  pred = best.predict([genres])[0]
  # print(pred)
  res = []
  for (ind, pred) in enumerate(pred):
      res.append((ind, pred))

  res.sort(key=lambda x: x[1], reverse=True)
  res = [(movie_enc.inverse_transform([id])[0], pred) for (id, pred) in res]
  for (movieId, pred) in res[0:n]:
    print(f"{movies_ind_to_name[movieId]} -> {pred}")
  return None

In [159]:
# godfather, taxidriver, raging bull, goodfellas
get_result([858, 111, 1228, 1213], 10)

Shawshank Redemption, The (1994) -> 0.4859429895877838
Godfather, The (1972) -> 0.43963679671287537
Pulp Fiction (1994) -> 0.42580491304397583
Silence of the Lambs, The (1991) -> 0.3379665017127991
Godfather: Part II, The (1974) -> 0.33401304483413696
Schindler's List (1993) -> 0.3322215676307678
Memento (2000) -> 0.31244343519210815
Monty Python and the Holy Grail (1975) -> 0.3116236925125122
American Beauty (1999) -> 0.3053744435310364
Casablanca (1942) -> 0.3002318739891052


In [160]:
# eat pray love, Eternal Sunshine of the Spotless Mind, the intern, The notebook
get_result([80551, 7361, 140110, 8533], 10)

Shawshank Redemption, The (1994) -> 0.4891999661922455
Godfather, The (1972) -> 0.4533776640892029
Pulp Fiction (1994) -> 0.43185603618621826
Silence of the Lambs, The (1991) -> 0.3504878878593445
Schindler's List (1993) -> 0.3471277356147766
Godfather: Part II, The (1974) -> 0.34486883878707886
Casablanca (1942) -> 0.3293911814689636
Monty Python and the Holy Grail (1975) -> 0.32638031244277954
Memento (2000) -> 0.320480614900589
American Beauty (1999) -> 0.31252288818359375


In [161]:
# into the wild, seven years in tibet, cast away, the pursuit of happyness
get_result([55247, 1619, 4022, 8131], 10)

Shawshank Redemption, The (1994) -> 0.4622631072998047
Pulp Fiction (1994) -> 0.38848093152046204
Godfather, The (1972) -> 0.3879435062408447
Matrix, The (1999) -> 0.363897442817688
Star Wars: Episode V - The Empire Strikes Back (1980) -> 0.3619690537452698
Terminator 2: Judgment Day (1991) -> 0.3266291916370392
Star Wars: Episode IV - A New Hope (1977) -> 0.3153918981552124
Silence of the Lambs, The (1991) -> 0.3139417767524719
Lord of the Rings: The Fellowship of the Ring, The (2001) -> 0.30483436584472656
Memento (2000) -> 0.3030233383178711
